In [18]:
!pip install langchain chromadb transformers google-cloud-aiplatform google-auth google-search-results > /dev/null

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/generativeaitrial-trialLC.json'

In [48]:
from langchain.llms import VertexAI
from langchain.chat_models import ChatVertexAI

In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

llm = VertexAI(temperature=0)
summary_chain = load_summarize_chain(llm, 
                                     chain_type="map_reduce")

In [4]:
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [ ]:
!unzip lc_documentdb.zip

In [6]:

embeddings = VertexAIEmbeddings()

persist_directory = '/content/content/lc_documentdb'

vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)

db_retriever = vectordb.as_retriever()


langchain_qa = RetrievalQA.from_chain_type(llm=VertexAI(), 
                                 chain_type="stuff", 
                                 retriever=db_retriever)

In [43]:
def summarize_output(topic):
  relevant_data = langchain_qa(topic)
  print(relevant_data)
  summary = summarize_document_chain.run(relevant_data['result'])
  return summary

In [8]:
query = 'List the langchain Agents'

In [33]:
summarize_output(query)

'Action agents act without planning, while plan-and-execute agents plan before acting.'

In [24]:
qa_chain = load_qa_chain(llm, chain_type="stuff")

qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)


In [34]:
query1 = 'What are the concepts in Langchain'

In [ ]:
docs = db_retriever.get_relevant_documents(query1)
docs

In [36]:
text = ''
for t in docs:
  text = text + t.page_content

In [40]:
qa_chain_route = qa_document_chain.run(input_document=text, 
                      question=query1)

In [41]:
qa_chain_route

'Chain of Thought, Action Plan Generation, ReAct, Self-ask, Prompt Chaining, Memetic Proxy, Self Consistency, Inception, MemPrompt'

In [44]:
summarize_output(query1)

{'query': 'What are the concepts in Langchain', 'result': 'The concepts in Langchain are Chain of Thought, Action Plan Generation, ReAct, Self-ask, Prompt Chaining, Memetic Proxy, Self Consistency, Inception, and MemPrompt.'}


'Langchain is a language model that can generate text, translate languages, write creative content, and answer your questions.'

Forward-Looking Active REtrieval augmented generation (FLARE).

The idea is:

Start answering a question

If you start generating tokens the model is uncertain about, look up relevant documents

Use those documents to continue generating

Repeat until finished


Need three things:

An LLM to generate the answer

An LLM to generate hypothetical questions to use in retrieval

A retriever to use to look up answers for

In [46]:
import os
os.environ["SERPER_API_KEY"] = "a5029817748ec7119fba1bff69445b00d3b9c617a88b8eaf7dbb4470af9a7d1a"
import re

import numpy as np

from langchain.schema import BaseRetriever
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.schema import Document

In [47]:
class SerperSearchRetriever(BaseRetriever):
    def __init__(self, search):
        self.search = search
    
    def get_relevant_documents(self, query: str):
        return [Document(page_content=self.search.run(query))]
    
    async def aget_relevant_documents(self, query: str):
        raise NotImplemented
        
        
retriever = SerperSearchRetriever(GoogleSerperAPIWrapper())

In [49]:
from langchain.chains import FlareChain

flare = FlareChain.from_llm(
    ChatVertexAI(temperature=0), 
    retriever=retriever,
    max_generation_len=164,
    min_prob=.3,
)

ValidationError: ignored